In [2]:
!nvidia-smi

Sun Mar  3 18:51:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8               4W /  80W |     14MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Imports

In [3]:
!pip install -q accelerate -U
!pip install -q simpletransformers

In [4]:
cd ..

/home/andyfu/Desktop/nlp-coursework-1


/home/andyfu/miniconda3/envs/nlp/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from ast import literal_eval
import numpy as np
from dont_patronize_me import DontPatronizeMe
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import torch.functional as F
import tqdm
import nltk
from nltk.corpus import wordnet
from torch import nn
nltk.download("punkt")
nltk.download("wordnet")

2024-03-03 18:52:01.197399: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-03 18:52:01.219446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 18:52:01.219477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 18:52:01.220055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 18:52:01.224113: I tensorflow/core/platform/cpu_feature_guar

True

In [5]:
random_seed = 42

In [6]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

device = 'cuda' if cuda_available else 'cpu'

print('Cuda available?', cuda_available)

Cuda available? True


# Data loading

In [7]:
def load_data(train_size=0.8, random_state=random_seed):
    dpm = DontPatronizeMe('.', '.')
    dpm.load_task1()
    trids = pd.read_csv('data/train_semeval_parids-labels.csv')
    teids = pd.read_csv('data/dev_semeval_parids-labels.csv')
    trids.par_id = trids.par_id.astype(str)
    teids.par_id = teids.par_id.astype(str)
    data = dpm.train_task1_df

    rows_train_val = [] # will contain par_id, label and text
    for idx in range(len(trids)):
        parid = trids.par_id[idx]
        # select row from original dataset to retrieve `text` and binary label
        instance = data.loc[data.par_id == parid]
        keyword = instance.keyword.values[0]
        text = instance.text.values[0]
        label = instance.label.values[0]
        rows_train_val.append({
            'text':text,
            'label':label
        })

    rows_train, rows_val = train_test_split(rows_train_val, train_size=train_size, random_state=random_state)

    rows_test = [] # will contain par_id, label and text
    for idx in range(len(teids)):
        parid = teids.par_id[idx]
        #print(parid)
        # select row from original dataset
        instance = data.loc[data.par_id == parid]
        keyword = instance.keyword.values[0]
        text = instance.text.values[0]
        label = instance.label.values[0]
        rows_test.append({
            'text':text,
            'label':label
        })
    print(len(rows_train), len(rows_val), len(rows_test))

    return pd.DataFrame(rows_train), pd.DataFrame(rows_val), pd.DataFrame(rows_test)

train_dataset_raw, eval_dataset_raw, test_dataset_raw = load_data(train_size=0.8)

6700 1675 2094


In [8]:
train_dataset_raw = pd.read_csv('data/train_set.csv')
eval_dataset_raw = pd.read_csv('data/eval_set.csv')

**Up-sample and Down-sample**

In [9]:
# train_dataset_positive = train_dataset_raw[train_dataset_raw.label == 1]
# train_dataset_raw = pd.concat([train_dataset_raw, train_dataset_positive])

In [10]:
# train_dataset_raw_negative = train_dataset_raw[train_dataset_raw.label == 0].sample(frac=0.7)
# train_dataset_raw_positive = train_dataset_raw[train_dataset_raw.label == 1]
# train_dataset_raw = pd.concat([train_dataset_raw_negative, train_dataset_raw_positive])
# train_dataset_raw

**Back Translation**

In [11]:
# from googletrans import Translator
# def backtranslate(text, target_language='zh-cn'):
#     translator = Translator()
    
#     # Translate to target language
#     translated = translator.translate(text, dest=target_language).text
    
#     # Translate back to original language
#     back_translated = translator.translate(translated, dest='en').text
    
#     return back_translated

# positives = train_dataset_raw[train_dataset_raw.label == 1]
# backtranslated = []
# for idx, row in positives.iterrows():
#     # if idx < 121:
#     #     continue
#     print(idx)
#     backtranslated.append(backtranslate(row.text))

# # add backtranslated positives to train dataset
# backtranslated = pd.DataFrame(backtranslated, columns=['text'])
# backtranslated['label'] = 1
# train_dataset_raw = pd.concat([train_dataset_raw, backtranslated])

# train_dataset_raw = pd.read_csv('data/back_trans_train.csv')
# eval_dataset_raw = pd.read_csv('data/back_trans_val.csv')

**Replacement**

In [12]:
# def replace_sentence_with_synonyms(sentence):
#     words = nltk.word_tokenize(sentence)
#     upper_case = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
#     synonyms = []
#     for word in words:
#         # if word contains capital letters, skip
#         if any([c in upper_case for c in word]):
#             synonyms.append(word)
#             continue
#         if np.random.rand() > 0.9 or len(word) <= 3:
#             synonyms.append(word)
#             continue
#         syns = wordnet.synsets(word)
#         if len(syns) > 0:
#             synonyms.append(syns[0].lemmas()[0].name())
#         else:
#             synonyms.append(word)
#     # convert back to sentence
#     sentence = ' '.join(synonyms)
#     return sentence

# # get a sample from the training dataset
# train_dataset_raw_positive = train_dataset_raw[train_dataset_raw.label == 1]
# sample = train_dataset_raw_positive.sample(1)
# print(sample.text.values[0])
# print(replace_sentence_with_synonyms(sample.text.values[0]))

In [13]:
# for idx, row in train_dataset_raw.iterrows():
#     train_dataset_raw.at[idx, 'text'] = replace_sentence_with_synonyms(row.text)
# train_dataset_raw = pd.concat([train_dataset_raw, train_dataset_raw_positive])

**Contextual embedding replacement**

In [14]:
import gensim
import transformers
import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action
from nlpaug.util.file.download import DownloadUtil
# initialising the augmentor with "Glove"
caug = naw.ContextualWordEmbsAug(
        # option to choose from is "word2vec", "glove" or "fasttext"
        model_path='distilroberta-base',

        # options available are insert or substitute
        action='substitute')

In [39]:
train_dataset_raw_positive = train_dataset_raw[train_dataset_raw.label == 1]

for idx, row in train_dataset_raw_positive.iterrows():
    train_dataset_raw.at[idx, 'text'] = caug.augment(row.text)

train_dataset_raw = pd.concat([train_dataset_raw, train_dataset_raw_positive])
print(train_dataset_raw)

                                                   text  label
0     Critics have even taken to dobbing in Katrina ...      0
1     Alexis and her family decided to donate more t...      0
2     [Mr Porter. do you think parliament should hea...      1
3     """ This only serves to highlight the importan...      0
4     """ I was nervous , but life has taught me ove...      0
...                                                 ...    ...
6645  President Uhuru Kenyatta and his deputy Willia...      1
6657  "Interacting with media , Adityanath said , ""...      1
6674  """ The United States respects and admires you...      1
6688  "An organisation representing Kashmiri Pandits...      1
6693  I have been interested and passionate about wh...      1

[7327 rows x 2 columns]


In [42]:
# save train_dataset_raw
train_dataset_raw.to_csv('data/augmented_train.csv', index=False)
train_dataset_raw = pd.read_csv('data/augmented_train.csv')

# Encode data into a Dataset

In [43]:
# Load the DeBERTa tokenizer
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

max_length = 192
batch_size = 8

train_text = train_dataset_raw.text.values
eval_text = eval_dataset_raw.text.values
test_text = test_dataset_raw.text.values


encoding_train = tokenizer(train_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)
encoding_eval = tokenizer(eval_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)
encoding_test = tokenizer(test_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)

encoding_train['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in train_dataset_raw['label'].tolist()], dtype=torch.float32)
encoding_eval['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in eval_dataset_raw['label'].tolist()], dtype=torch.float32)
encoding_test['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in test_dataset_raw['label'].tolist()], dtype=torch.float32)

In [44]:
class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        train_input_ids = self.encodings.input_ids[idx]
        train_token_type_ids = self.encodings.token_type_ids[idx]
        train_attention_mask = self.encodings.attention_mask[idx]
        train_labels = self.encodings.label[idx]
        return {
            'input_ids': train_input_ids,
            'token_type_ids': train_token_type_ids,
            'attention_mask': train_attention_mask,
            'labels': train_labels
        }

    def __len__(self):
        return len(self.encodings.input_ids)

In [45]:
# Create an instance of the CustomDataset class
train_dataset = CustomDataset(encoding_train)
eval_dataset = CustomDataset(encoding_eval)
test_dataset = CustomDataset(encoding_test)

# Model loading and training

In [52]:
class PatronizeTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs = False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.nn.functional.softmax(logits, dim=1)
        weights = torch.tensor([0.7, 1]).to(device)
        loss = nn.BCELoss(weight=weights)(predictions, labels)

        return (loss, outputs) if return_outputs else loss

In [53]:
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)
model = model.to(device)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
epochs = 8
lr = 1e-5

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    labels = np.argmax(labels, axis=-1)
    f1 = f1_score(labels, predictions, average='binary')
    return {'f1': f1}

trainingargs = TrainingArguments(
    learning_rate=lr,
    weight_decay=1e-2,
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    report_to=None,
    metric_for_best_model="f1",
    save_strategy='epoch',
    load_best_model_at_end=True,
    seed=random_seed,
    optim='adamw_torch',
)

trainer = PatronizeTrainer(
    model=model,
    args=trainingargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
# Here the best model has been already loaded
trainer.predict(test_dataset)

  0%|          | 0/7328 [00:00<?, ?it/s]

{'loss': 0.2754, 'grad_norm': 14.37192153930664, 'learning_rate': 9.317685589519652e-06, 'epoch': 0.55}


  0%|          | 0/210 [00:00<?, ?it/s]

Checkpoint destination directory ./results/checkpoint-916 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.20147934556007385, 'eval_f1': 0.49295774647887325, 'eval_runtime': 12.4894, 'eval_samples_per_second': 134.113, 'eval_steps_per_second': 16.814, 'epoch': 1.0}
{'loss': 0.2059, 'grad_norm': 3.310044527053833, 'learning_rate': 8.635371179039303e-06, 'epoch': 1.09}
{'loss': 0.1762, 'grad_norm': 0.9693918824195862, 'learning_rate': 7.953056768558953e-06, 'epoch': 1.64}


...

  0%|          | 0/210 [00:00<?, ?it/s]

Checkpoint destination directory ./results/checkpoint-7328 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.611011266708374, 'eval_f1': 0.5205479452054794, 'eval_runtime': 12.5745, 'eval_samples_per_second': 133.206, 'eval_steps_per_second': 16.7, 'epoch': 8.0}
{'train_runtime': 1703.0456, 'train_samples_per_second': 34.418, 'train_steps_per_second': 4.303, 'train_loss': 0.07645877031812762, 'epoch': 8.0}


  0%|          | 0/262 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ 1.5687085, -1.7673488],
       [-1.5307105,  1.7029703],
       [ 2.222961 , -2.3399305],
       ...,
       [ 2.5631595, -2.7579694],
       [ 1.7213833, -1.8195697],
       [ 3.4756799, -3.5509446]], dtype=float32), label_ids=array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32), metrics={'test_loss': 0.20992513000965118, 'test_f1': 0.5729166666666666, 'test_runtime': 15.617, 'test_samples_per_second': 134.085, 'test_steps_per_second': 16.777})

In [29]:
trainer.save_model('./results/best_model')

# Saving predictions

In [12]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')